In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import cross_validation as cv

In [62]:
BASE  = './bytecup2016data'
IINFO = BASE + '/invited_info_train.txt'
QINFO = BASE + '/question_info.txt'
UINFO = BASE + '/user_info.txt'
VAL   = BASE + '/validate_nolabel.txt'

invdata = pd.read_csv(IINFO, delim_whitespace=True, header=None, names=["qid", "uid", "answered"])
qdata   = pd.read_csv(QINFO, delim_whitespace=True, header=None, names=["qid", "qtag", "wseq", "cseq", "nvotes", "nans", "ntqans"])
udata   = pd.read_csv(UINFO, delim_whitespace=True, header=None, names=["uid", "exptag", "wseq", "cseq"])
valdata = pd.read_csv(VAL)

# merged_data = qdata.merge(train_invdata,on="qid", how="left").merge(udata, on="uid", how="right") 
# ratings_mtx_df = merged_data.pivot_table(values='answered',
#                                              index='uid',
#                                              columns='qid')

In [63]:
invdata = invdata[(invdata.answered == 1)]

In [64]:
training_qid = invdata['qid'].tolist()
training_uid = invdata['uid'].tolist()

In [65]:
all_uids = udata['uid'].tolist()
all_qids = qdata['qid'].tolist()

In [66]:
n_train = len(training_qid)
train_data = []
for i in range(n_train):
    q_pos = all_qids.index(training_qid[i])
    u_pos = all_uids.index(training_uid[i])
    train_data.append((u_pos,q_pos))

In [67]:
from theano_bpr import BPR

In [68]:
bpr = BPR(10, len(all_uids), len(all_qids))

In [69]:
bpr.train(train_data, epochs=50)

Generating 1366200 random training samples
Processed 1366000 ( 99.99% ) in 0.0140 seconds
Total training time 20.09 seconds; 1.470498e-05 per sample


In [70]:
testing_qid = valdata['qid'].tolist()
testing_uid = valdata['uid'].tolist()

In [71]:
valdata["label"] = ""

In [72]:
valdata=valdata.reset_index()

In [74]:
n_test = len(testing_qid)
test_data = []
for i in range(n_test):
    q_pos = all_qids.index(testing_qid[i])
    u_pos = all_uids.index(testing_uid[i])
    prediction = bpr.prediction(u_pos,q_pos)
    valdata.iloc[i, valdata.columns.get_loc('label')] = prediction

In [76]:
valdata.to_csv('bpr_results.csv')